In [1]:
# HMM smoother

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from hmmlearn import hmm

from wrangle_updated import trial, set_state
from utilities import convert_to_words, get_position_stats, combine_csv, resolve_acc_gyro, blank_filter, concat_data

TIME_SEQUENCE_LENGTH = 30
DIR = os.path.dirname(os.path.realpath('__file__'))

In [93]:
# Moving all the data prep functions here to allow for experimentation

In [94]:
def rolling_average(df):
    return pd.rolling_mean(df, window=TIME_SEQUENCE_LENGTH-2, center=True).mean()

def rolling_median(df):
    return pd.rolling_median(df, window=TIME_SEQUENCE_LENGTH-2, center=True).mean()

def rolling_max(df):
    return pd.rolling_max(df, window=TIME_SEQUENCE_LENGTH-2, center=True).mean()

def rolling_min(df):
    return pd.rolling_min(df, window=TIME_SEQUENCE_LENGTH-2, center=True).mean()

def standard_deviation(df):
    std = pd.rolling_std(df, window=TIME_SEQUENCE_LENGTH-2, center=True).mean()
    return std

def max_min_dif(df):
    diff = (pd.rolling_min(df, window=TIME_SEQUENCE_LENGTH-2, center=True).mean()) - (pd.rolling_max(df, window=TIME_SEQUENCE_LENGTH-2, center=True).mean())
    return diff
    

def create_rm_feature(df, sequence_length):
    
    # Need to find a way to create windows
    
    features = []

    original_x = df['ACCEL_X'].astype(float)
    original_y = df['ACCEL_Y'].astype(float)
    original_z = df['ACCEL_Z'].astype(float)
    original_gx = df['GYRO_X'].astype(float)
    original_gy = df['GYRO_Y'].astype(float)
    original_gz = df['GYRO_Z'].astype(float)

    i = original_x.index.values
    time_sequence = [i] * sequence_length

    # The acutal number of 'i' values in the np.array is the sequence_length

    # COULD try putting a Kalman filter here before we split into 10 second chunks
    # this would smooth all the readings before processing - not sure if that is a good approach...
    # This filter could replace values or form new features

    idx = np.array(time_sequence).T.flatten()[:len(original_x)]
    
    print "IDX"
    print idx
    
    x = original_x.groupby(idx).mean()
    print x
    x.name = 'ACCEL_X'
    features.append(x)

    y = original_y.groupby(idx).mean()
    y.name = 'ACCEL_Y'
    features.append(y)

    z = original_z.groupby(idx).mean()
    z.name = 'ACCEL_Z'
    features.append(z)

    gx = original_gx.groupby(idx).mean()
    gx.name = 'GYRO_X'
    features.append(gx)

    gy = original_gy.groupby(idx).mean()
    gy.name = 'GYRO_Y'
    features.append(gy)

    gz = original_gz.groupby(idx).mean()
    gz.name = 'GYRO_Z'
    features.append(gz)
    
    #rolling median

    x_ra = df['ACCEL_X'].groupby(idx).apply(rolling_median)
    x_ra.name = 'rolling_median_x'
    features.append(x_ra)

    y_ra = df['ACCEL_Y'].groupby(idx).apply(rolling_median)
    y_ra.name = 'rolling_median_y'
    features.append(y_ra)

    z_ra = df['ACCEL_Z'].groupby(idx).apply(rolling_median)
    z_ra.name = 'rolling_median_z'
    features.append(z_ra)

    #gx_ra = df['GYRO_X'].groupby(idx).apply(rolling_median)
    #gx_ra.name = 'rolling_median_gx'
    #features.append(gx_ra)

    #gy_ra = df['GYRO_Y'].groupby(idx).apply(rolling_median)
    #gy_ra.name = 'rolling_median_gy'
    #features.append(gy_ra)

    #gz_ra = df['GYRO_Z'].groupby(idx).apply(rolling_median)
    #gz_ra.name = 'rolling_median_gz'
    #features.append(gz_ra)

    #rolling max

    x_rm = df['ACCEL_X'].groupby(idx).apply(rolling_max)
    x_rm.name = 'rolling_max_x'
    features.append(x_rm)

    y_rm = df['ACCEL_Y'].groupby(idx).apply(rolling_max)
    y_rm.name = 'rolling_max_y'
    features.append(y_rm)

    z_rm = df['ACCEL_Z'].groupby(idx).apply(rolling_max)
    z_rm.name = 'rolling_max_z'
    features.append(z_rm)

    #gx_rm = df['GYRO_X'].groupby(idx).apply(rolling_max)
    #gx_rm.name = 'rolling_max_gx'
    #features.append(gx_rm)

    #gy_rm = df['GYRO_Y'].groupby(idx).apply(rolling_max)
    #gy_rm.name = 'rolling_max_gy'
    #features.append(gy_rm)

    #gz_rm = df['GYRO_Z'].groupby(idx).apply(rolling_max)
    #gz_rm.name = 'rolling_max_gz'
    #features.append(gz_rm)

    #rolling min

    x_rmin = df['ACCEL_X'].groupby(idx).apply(rolling_min)
    x_rmin.name = 'rolling_min_x'
    features.append(x_rmin)

    y_rmin = df['ACCEL_Y'].groupby(idx).apply(rolling_min)
    y_rmin.name = 'rolling_min_y'
    features.append(y_rmin)

    z_rmin = df['ACCEL_Z'].groupby(idx).apply(rolling_min)
    z_rmin.name = 'rolling_min_z'
    features.append(z_rmin)

    #gx_rmin = df['GYRO_X'].groupby(idx).apply(rolling_min)
    #gx_rmin.name = 'rolling_min_gx'
    #features.append(gx_rmin)

    #gy_rmin = df['GYRO_Y'].groupby(idx).apply(rolling_min)
    #gy_rmin.name = 'rolling_min_gy'
    #features.append(gy_rmin)

    #gz_rmin = df['GYRO_Z'].groupby(idx).apply(rolling_min)
    #gz_rmin.name = 'rolling_min_gz'
    #features.append(gz_rmin)

    #standard deviation

    x_std = df['ACCEL_X'].groupby(idx).apply(standard_deviation)
    x_std.name = 'std_x'
    features.append(x_std)

    y_std = df['ACCEL_Y'].groupby(idx).apply(standard_deviation)
    y_std.name = 'std_y'
    features.append(y_std)

    z_std = df['ACCEL_Z'].groupby(idx).apply(standard_deviation)
    z_std.name = 'std_z'
    features.append(z_std)

    gx_std = df['GYRO_X'].groupby(idx).apply(standard_deviation)
    gx_std.name = 'std_gx'
    features.append(gx_std)

    gy_std = df['GYRO_Y'].groupby(idx).apply(standard_deviation)
    gy_std.name = 'std_gy'
    features.append(gy_std)

    gz_std = df['GYRO_Z'].groupby(idx).apply(standard_deviation)
    gz_std.name = 'std_gz'
    features.append(gz_std)

    # Max min diff

    x_diff = df['ACCEL_X'].groupby(idx).apply(max_min_dif)
    x_diff.name = 'diff_x'
    features.append(x_diff)

    y_diff = df['ACCEL_Y'].groupby(idx).apply(max_min_dif)
    y_diff.name = 'diff_y'
    features.append(y_diff)

    z_diff = df['ACCEL_Z'].groupby(idx).apply(max_min_dif)
    z_diff.name = 'diff_z'
    features.append(z_diff)

    gx_diff = df['GYRO_X'].groupby(idx).apply(max_min_dif)
    gx_diff.name = 'diff_gx'
    features.append(gx_diff)

    gy_diff = df['GYRO_Y'].groupby(idx).apply(max_min_dif)
    gy_diff.name = 'diff_gy'
    features.append(gy_diff)

    gz_diff = df['GYRO_Z'].groupby(idx).apply(max_min_dif)
    gz_diff.name = 'diff_gz'
    features.append(gz_diff)

    data = pd.concat(features, axis=1)
    return data

def root_sum_square(x, y, z):
        sum = ((x**2)+(y**2)+(z**2))
        rss = math.sqrt(sum)
        return rss

def create_rss(df):
    df['acc_rss'] = df.apply(lambda x: root_sum_square(x['ACCEL_X'], x['ACCEL_Y'], x['ACCEL_Z']), axis=1)
    df['gyro_rss'] = df.apply(lambda x: root_sum_square(x['GYRO_X'], x['GYRO_Y'], x['GYRO_Z']), axis=1)
    return df
    
    

In [95]:
def combine_setState_createFeatures(directory, state):
    """
    convenience method to combine three steps in one function:
    (1) combine multiple csv files, (2) set their movement state for training,
    (3) combine to create time sequences and add features
    """
    combined_data = combine_csv(directory)
    combined_data_updated = set_state(combined_data, state) # TO CHECK: probably not necessary 
    feature_training_data = create_rm_feature(combined_data_updated, TIME_SEQUENCE_LENGTH)
    
    feature_training_data2 = create_rss(feature_training_data)
    ready_training_data = set_state(feature_training_data, state)
    return ready_training_data

In [96]:
def prep():
    """prepare the raw sensor data"""

    #1 Your mount
    ymount_td = combine_setState_createFeatures('your_mount_raw_data', 'your_mount')
    #2 Your side control
    ysc_td = combine_setState_createFeatures('your_side_control_raw_data', 'your_side_control')
    #3 Your closed guard
    ycg_td = combine_setState_createFeatures('your_closed_guard_raw_data', 'your_closed_guard')
    #4 Your back control
    ybc_td = combine_setState_createFeatures('your_back_control_raw_data', 'your_back_control')
    #5 Opponent mount or opponent side control
    omountsc_td = combine_setState_createFeatures('opponent_mount_and_opponent_side_control_raw_data', 'opponent_mount_or_sc')
    #6 Opponent closed guard
    ocg_td = combine_setState_createFeatures('opponent_closed_guard_raw_data', 'opponent_closed_guard')
    #7 Opponent back control
    obc_td = combine_setState_createFeatures('opponent_back_control_raw_data', 'opponent_back_control')
    #8 "Non jiu-jitsu" motion
    nonjj_td = combine_setState_createFeatures('non_jj_raw_data', 'non_jj')

    training_data = concat_data([ymount_td, ysc_td, ycg_td, ybc_td, omountsc_td, ocg_td, obc_td, nonjj_td])
    # remove NaN
    training_data = blank_filter(training_data)
    return training_data

In [106]:
def prep_test(el_file):
    el_file = 'data/test_cases/' + el_file
    df = pd.DataFrame()
    df = pd.read_csv(el_file, index_col=None, header=0)
    df = resolve_acc_gyro(df)
    df = create_rm_feature(df, TIME_SEQUENCE_LENGTH)
    df2 = create_rss(df)
    test_data = blank_filter(df2)

    return test_data

In [98]:
training_data = prep()

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_CS.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymount.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs2.csv']
IDX
[  0   0   0 ..., 498 498 498]
0     -0.956200
1     -0.967133
2     -0.979100
3     -1.011567
4     -1.015500
5     -1.014967
6     -1.003933
7     -1.012967
8     -1.009600
9     -1.011600
10    -1.011767
11    -1.012433
12    -1.014500
13    -1.007733
14    -1.012033
15    -1.014367
16    -1.008667
17    -1.022667
18    -1.013467
19    -1.013500
20    -1.008400
21    -0.995500
22    -0.877567
23    -0.640967
24    -0.583133
25    -0.583167
26    -0.784300
27   

In [99]:
print training_data.acc_rss
print training_data.ACCEL_X

0       1.026252
1       1.023676
2       1.017012
3       1.025399
4       1.026710
5       1.027768
6       1.023534
7       1.027423
8       1.026074
9       1.027802
10      1.026747
11      1.026755
12      1.027208
13      1.023044
14      1.023721
15      1.020246
16      1.024087
17      1.024428
18      1.021615
19      1.020699
20      1.024729
21      1.017279
22      1.014254
23      1.002729
24      1.008167
25      1.008081
26      0.987814
27      1.014277
28      1.012388
29      1.012911
          ...   
4174    1.023417
4175    1.021417
4176    1.022300
4177    1.023152
4178    1.024339
4179    1.020212
4180    1.024679
4181    1.024024
4182    1.015193
4183    1.021059
4184    1.003578
4185    1.023765
4186    1.023280
4187    1.024103
4188    1.023435
4189    1.025013
4190    1.024817
4191    1.022956
4192    1.017155
4193    0.980762
4194    1.023394
4195    0.980048
4196    0.997582
4197    1.023476
4198    1.025407
4199    1.023924
4200    1.024802
4201    1.0251

In [100]:
print training_data

      index   ACCEL_X   ACCEL_Y   ACCEL_Z     GYRO_X     GYRO_Y     GYRO_Z  rolling_median_x  rolling_median_y  rolling_median_z  ...       std_gz    diff_x    diff_y    diff_z    diff_gx    diff_gy    diff_gz   acc_rss   gyro_rss  state
0         0 -0.956200 -0.138667  0.345900   1.638267  -3.331367   2.634300         -0.959000         -0.144333          0.343000  ...     1.476862 -0.031000 -0.082667 -0.086333 -16.768000 -13.536000  -5.874333  1.026252   4.552083      0
1         1 -0.967133 -0.141233  0.304333  -0.987900   4.613767  -2.477667         -0.967167         -0.140667          0.293833  ...     5.242716 -0.071000 -0.095000 -0.130333 -37.641667 -18.191000 -17.011667  1.023676   5.329317      0
2         2 -0.979100 -0.275000  0.007133 -15.534633  13.845567   2.855633         -0.991333         -0.299833         -0.040000  ...    13.456694 -0.154000 -0.239667 -0.449667 -62.927000 -39.634000 -39.513000  1.017012  21.004266      0
3         3 -1.011567 -0.164133 -0.035167   7.92

In [101]:
#1 Your mount
ymount_td = combine_setState_createFeatures('your_mount_raw_data', 'your_mount')
#2 Your side control
ysc_td = combine_setState_createFeatures('your_side_control_raw_data', 'your_side_control')
#3 Your closed guard
ycg_td = combine_setState_createFeatures('your_closed_guard_raw_data', 'your_closed_guard')
#4 Your back control
ybc_td = combine_setState_createFeatures('your_back_control_raw_data', 'your_back_control')
#5 Opponent mount or opponent side control
omountsc_td = combine_setState_createFeatures('opponent_mount_and_opponent_side_control_raw_data', 'opponent_mount_or_sc')
#6 Opponent closed guard
ocg_td = combine_setState_createFeatures('opponent_closed_guard_raw_data', 'opponent_closed_guard')
#7 Opponent back control
obc_td = combine_setState_createFeatures('opponent_back_control_raw_data', 'opponent_back_control')
#8 "Non jiu-jitsu" motion
nonjj_td = combine_setState_createFeatures('non_jj_raw_data', 'non_jj')

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_CS.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymount.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs2.csv']
IDX
[  0   0   0 ..., 498 498 498]
0     -0.956200
1     -0.967133
2     -0.979100
3     -1.011567
4     -1.015500
5     -1.014967
6     -1.003933
7     -1.012967
8     -1.009600
9     -1.011600
10    -1.011767
11    -1.012433
12    -1.014500
13    -1.007733
14    -1.012033
15    -1.014367
16    -1.008667
17    -1.022667
18    -1.013467
19    -1.013500
20    -1.008400
21    -0.995500
22    -0.877567
23    -0.640967
24    -0.583133
25    -0.583167
26    -0.784300
27   

In [102]:
print ymount_td

      ACCEL_X   ACCEL_Y   ACCEL_Z     GYRO_X     GYRO_Y     GYRO_Z  rolling_median_x  rolling_median_y  rolling_median_z  rolling_max_x  ...       std_gz    diff_x    diff_y    diff_z     diff_gx     diff_gy     diff_gz   acc_rss   gyro_rss  state
0   -0.956200 -0.138667  0.345900   1.638267  -3.331367   2.634300         -0.959000         -0.144333          0.343000      -0.937000  ...     1.476862 -0.031000 -0.082667 -0.086333  -16.768000  -13.536000   -5.874333  1.026252   4.552083      0
1   -0.967133 -0.141233  0.304333  -0.987900   4.613767  -2.477667         -0.967167         -0.140667          0.293833      -0.934000  ...     5.242716 -0.071000 -0.095000 -0.130333  -37.641667  -18.191000  -17.011667  1.023676   5.329317      0
2   -0.979100 -0.275000  0.007133 -15.534633  13.845567   2.855633         -0.991333         -0.299833         -0.040000      -0.903000  ...    13.456694 -0.154000 -0.239667 -0.449667  -62.927000  -39.634000  -39.513000  1.017012  21.004266      0
3   -1.0

In [107]:
test_data1 = prep_test('test1_ymount_ycg.csv')
test_data4 = prep_test('GL_TEST1_CS.csv')
test_data5 = prep_test('GL_TEST2_CS.csv')
test_data6 = prep_test('GL_TEST3_CS_very_still.csv')
test_data7 = prep_test('GL_TEST1_UrsWearing.csv')

print training_data[0:1]
print test_data1[0:1]


IDX
[ 0  0  0 ..., 41 42 42]
0    -0.973200
1    -0.964100
2    -1.012933
3    -0.754667
4    -0.523333
5    -0.954067
6    -0.944567
7    -0.961867
8    -0.753233
9    -0.932800
10   -0.822767
11   -0.929867
12   -0.981100
13   -0.734100
14   -0.953700
15   -0.940133
16   -0.986933
17   -0.565900
18   -0.019733
19    0.151367
20    0.087967
21    0.012967
22   -0.113367
23   -0.080333
24    0.059567
25   -0.012567
26   -0.071767
27    0.092833
28    0.091167
29   -0.041767
30   -0.258200
31   -0.182833
32    0.100033
33    0.055300
34   -0.076400
35    0.118933
36    0.250967
37    0.243000
38    0.256667
39    0.273167
40    0.291900
41    0.249185
42         NaN
Name: ACCEL_X, dtype: float64
Removed 2 NaN rows
IDX
[ 0  0  0 ..., 81 82 82]
0    -1.014633
1    -0.876633
2    -0.515733
3    -0.199833
4    -0.159567
5    -0.153900
6    -0.167000
7    -0.010767
8    -0.815800
9    -0.959600
10   -0.691633
11   -0.952167
12   -0.860500
13   -0.889600
14   -0.754433
15   -0.602900
16   -0.

In [108]:
pre_smooth = trial(training_data, test_data1)

[7 7 7 0 0 0 5 5 0 5 0 5 5 0 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
['OTHER', 'OTHER', 'OTHER', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc']
Your Mount: 0.170731707317
Your Side Control: 0.0
Your Closed Guard: 0.341463414634
Your Back 

In [109]:
pre_smooth2 = trial(training_data, test_data4)

[7 7 3 6 3 3 3 3 3 0 0 0 0 5 7 0 0 1 1 0 1 6 1 1 1 1 0 7 4 4 4 4 2 3 4 4 2
 2 2 2 2 4 2 5 0 0 5 5 5 5 0 5 5 5 2 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 4 6 6 6
 6 6 6 6 6 1 0]
['OTHER', 'OTHER', 'your_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'OTHER', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_mount', 'your_side_control', 'opponent_back_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'OTHER', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_closed_guard', 'your_back_control', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', '

In [9]:
#pre_smooth3 = trial(training_data, test_data5)

[7 7 6 1 6 6 1 6 6 6 3 3 3 1 0 5 0 0 0 0 1 1 1 1 1 1 1 0 4 2 2 2 2 2 2 2 2
 2 2 2 7 0 5 5 0 5 5 5 5 5 5 5 5 7 4 4 4 4 2 4 2 4 4 4 2 4 4 4 4 2 4 3 6 2
 4 6 3 4 6 6 6 6 6 6 6 6 6 7 5 7]
['OTHER', 'OTHER', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_side_control', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'OTHER', '

In [10]:
#pre_smooth4 = trial(training_data, test_data6)

[7 7 7 6 3 3 6 6 6 3 3 3 5 0 0 5 5 5 5 5 5 7 7 7 7 5 7 5 0 1 1 1 1 1 1 1 1
 1 1 1 5 7 2 2 2 2 7 7 7 2 2 2 7 7 7 2 2 7 7 7 7 2 2 2 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 0 5 7 4 4 4 4 4 7 7 7 7 7 7 7 7 7 2 4 4 4 4 4 4 4 4 7 4 7 7 7 7
 4 4 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 7 5]
['OTHER', 'OTHER', 'OTHER', 'opponent_back_control', 'your_back_control', 'your_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_c

In [11]:
#pre_smooth5 = trial(training_data, test_data7)

[7 7 2 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 4 4 4 4 4 4 2 2 4 4
 1 0 0 0 3 0 0 0 5 0 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 6 3 6 6 6 6 6 6 6 6 6
 1 1 5 7]
['OTHER', 'OTHER', 'your_closed_guard', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_side_control', 'your_mount', 'your_mount', 'your_mount', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'oppon

In [110]:
print pre_smooth
pre_smooth_words = convert_to_words(pre_smooth)
pre_smooth_words2 = convert_to_words(pre_smooth2)
#pre_smooth_words3 = convert_to_words(pre_smooth3)
#pre_smooth_words4 = convert_to_words(pre_smooth4)
#pre_smooth_words5 = convert_to_words(pre_smooth5)
print pre_smooth_words

[7 7 7 0 0 0 5 5 0 5 0 5 5 0 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
['OTHER', 'OTHER', 'OTHER', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc']


In [111]:
n_components = 8 # ('ybc', 'ymount', 'ysc', 'ycg', 'ocg', 'osc_mount', 'obc', 'other')
# n_components = 3
startprob = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.65,]) # users will probably turn on sensor standing
# startprob = np.array([0.34, 0.33, 0.33])

In [112]:
# transmat = np.array([[0.34, 0.33, 0.33], [0.9, 0.05, 0.05], [0.9, 0.05, 0.05]])

"""
probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

transition_probability = {
        'ymt' : {'ymount': 0.800, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.800, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.800, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.001, 'osc_mount': 0.800, 'ocg': 0.050, 'obc': 0.050, 'other': 0.038},
        'ocg' : {'ymount': 0.100, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.800, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.800, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

transmat = np.array([
                    [0.800, 0.050, 0.010, 0.050, 0.001, 0.050, 0.001, 0.038], 
                    [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                    [0.010, 0.050, 0.800, 0.010, 0.050, 0.001, 0.001, 0.078], 
                    [0.050, 0.010, 0.050, 0.800, 0.001, 0.010, 0.001, 0.078],
                    [0.001, 0.050, 0.010, 0.001, 0.800, 0.050, 0.050, 0.038],
                    [0.100, 0.050, 0.010, 0.010, 0.001, 0.800, 0.001, 0.028],
                    [0.010, 0.050, 0.001, 0.010, 0.050, 0.001, 0.800, 0.078],
                    [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                    ])

In [113]:
# emissionprob = np.array([[0.34, 0.33, 0.33], [0.4, 0.55, 0.05], [0.05, 0.55, 0.4]])

"""

probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

emission_probability = {
        'ymt' : {'ymount': 0.500, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.350, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.400, 'ybc': 0.010, 'osc_mount': 0.500, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.600, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.201, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.210, 'ybc': 0.050, 'osc_mount': 0.600, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ocg' : {'ymount': 0.400, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.400, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.110, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.700, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

emissionprob = np.array([
                        [0.500, 0.050, 0.010, 0.050, 0.001, 0.350, 0.001, 0.038], 
                        [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                        [0.010, 0.050, 0.350, 0.010, 0.500, 0.001, 0.001, 0.078], 
                        [0.050, 0.010, 0.050, 0.700, 0.001, 0.010, 0.101, 0.078],
                        [0.001, 0.050, 0.210, 0.050, 0.600, 0.050, 0.001, 0.038],
                        [0.400, 0.050, 0.010, 0.010, 0.001, 0.400, 0.001, 0.028],
                        [0.010, 0.050, 0.001, 0.110, 0.050, 0.001, 0.700, 0.078],
                        [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                        ])


In [114]:
# Hidden Markov Model with multinomial (discrete) emissions
model = hmm.MultinomialHMM(n_components=n_components,
                           n_iter=10,
                           verbose=False)

model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob
# model.n_features = 8

In [115]:
# observations = np.array([1, 1, 2, 2, 1, 0, 1, 2, 2, 0])
observations = np.array(pre_smooth)
observations2 = np.array(pre_smooth2)
#observations3 = np.array(pre_smooth3)
#observations4 = np.array(pre_smooth4)
#observations5 = np.array(pre_smooth5)
a,b = model.sample(5)
print a,b
print '=========='

n_samples = len(observations)
data = observations.reshape((n_samples, -1))
print data

n_samples2 = len(observations2)
data2 = observations2.reshape((n_samples2, -1))

#n_samples3 = len(observations3)
#data3 = observations3.reshape((n_samples3, -1))

#n_samples4 = len(observations4)
#data4 = observations4.reshape((n_samples4, -1))

#n_samples5 = len(observations5)
#data5 = observations5.reshape((n_samples5, -1))

[[2]
 [2]
 [4]
 [4]
 [3]] [4 4 4 4 4]
[[7]
 [7]
 [7]
 [0]
 [0]
 [0]
 [5]
 [5]
 [0]
 [5]
 [0]
 [5]
 [5]
 [0]
 [5]
 [7]
 [5]
 [0]
 [6]
 [4]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [4]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [4]
 [4]
 [4]
 [4]
 [4]
 [4]]


In [116]:
# decode(X, lengths=None, algorithm=None)[source]
# Find most likely state sequence corresponding to X.
# Will work best for organic tests

"""correct sequence
your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7


[3, 0, 1, 2, 5, 4, 6]

"""

"correct sequence\nyour_mount' if v == 0 \nelse 'your_side_control' if v == 1\nelse 'your_closed_guard' if v == 2\nelse 'your_back_control' if v == 3\nelse 'opponent_mount_or_sc' if v == 4\nelse 'opponent_closed_guard' if v == 5\nelse 'opponent_back_control' if v == 6\nelse 'OTHER' if v == 7\n\n\n[3, 0, 1, 2, 5, 4, 6]\n\n"

In [117]:
print 'TEST 1'

result = model.decode(data, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth)
print 'result accuracy {}%'.format(result[0])
print 'final result: {}'.format(result[1])

result_words = convert_to_words(result[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words)
print '====================='
print 'result words: {}'.format(result_words)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words)

print '\n'

print 'result stats'
print get_position_stats(result_words)

print '******************'




TEST 1
pre smooth: [7 7 7 0 0 0 5 5 0 5 0 5 5 0 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
result accuracy -56.3499979936%
final result: [7 7 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2]
pre smooth words: ['OTHER', 'OTHER', 'OTHER', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc'

In [118]:
print 'TEST2'
result2 = model.decode(data2, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth2)
print 'result accuracy {}%'.format(result2[0])
print 'final result: {}'.format(result2[1])

result_words2 = convert_to_words(result2[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words2)
print '====================='
print 'result words: {}'.format(result_words2)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words2)

print '\n'

print 'result stats'
print get_position_stats(result_words2)

print '******************'

TEST2
pre smooth: [7 7 3 6 3 3 3 3 3 0 0 0 0 5 7 0 0 1 1 0 1 6 1 1 1 1 0 7 4 4 4 4 2 3 4 4 2
 2 2 2 2 4 2 5 0 0 5 5 5 5 0 5 5 5 2 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 4 6 6 6
 6 6 6 6 6 1 0]
result accuracy -120.080821923%
final result: [7 7 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 7 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 2 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 6 6 6 6
 6 6 6 6 6 1 1]
pre smooth words: ['OTHER', 'OTHER', 'your_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'OTHER', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_mount', 'your_side_control', 'opponent_back_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'OTHER', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_o

In [21]:
"""
print 'TEST3'
result3 = model.decode(data3, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth3)
print 'result accuracy {}%'.format(result3[0])
print 'final result: {}'.format(result3[1])

result_words3 = convert_to_words(result3[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words3)
print '====================='
print 'result words: {}'.format(result_words3)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words3)

print '\n'

print 'result stats'
print get_position_stats(result_words3)

print '******************'
"""

TEST3
pre smooth: [7 7 6 1 6 6 1 6 6 6 3 3 3 1 0 5 0 0 0 0 1 1 1 1 1 1 1 0 4 2 2 2 2 2 2 2 2
 2 2 2 7 0 5 5 0 5 5 5 5 5 5 5 5 7 4 4 4 4 2 4 2 4 4 4 2 4 4 4 4 2 4 3 6 2
 4 6 3 4 6 6 6 6 6 6 6 6 6 7 5 7]
result accuracy -142.448382567%
final result: [7 7 6 6 6 6 6 6 6 6 3 3 3 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 7 0 0 0 0 0 0 0 0 0 0 0 0 7 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 6 6 4
 4 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7]
pre smooth words: ['OTHER', 'OTHER', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_side_control', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount',

In [22]:
"""
print 'TEST4'
result4 = model.decode(data4, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth4)
print 'result accuracy {}%'.format(result4[0])
print 'final result: {}'.format(result4[1])

result_words4 = convert_to_words(result4[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words4)
print '====================='
print 'result words: {}'.format(result_words4)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words4)

print '\n'

print 'result stats'
print get_position_stats(result_words4)

print '******************'
"""

TEST4
pre smooth: [7 7 7 6 3 3 6 6 6 3 3 3 5 0 0 5 5 5 5 5 5 7 7 7 7 5 7 5 0 1 1 1 1 1 1 1 1
 1 1 1 5 7 2 2 2 2 7 7 7 2 2 2 7 7 7 2 2 7 7 7 7 2 2 2 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 0 5 7 4 4 4 4 4 7 7 7 7 7 7 7 7 7 2 4 4 4 4 4 4 4 4 7 4 7 7 7 7
 4 4 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 7 5]
result accuracy -190.670795411%
final result: [7 7 7 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 7 7 7 7 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 0 7 2 2 2 2 7 7 7 2 2 2 7 7 7 7 7 7 7 7 7 2 2 2 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 7 2 2 2 2 2 7 7 7 7 7 7 7 7 7 2 2 2 2 2 2 2 2 2 2 2 7 7 7 7
 2 4 4 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 0 0]
pre smooth words: ['OTHER', 'OTHER', 'OTHER', 'opponent_back_control', 'your_back_control', 'your_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'oppone

In [23]:
"""
print 'TEST5'
result5 = model.decode(data5, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth5)
print 'result accuracy {}%'.format(result5[0])
print 'final result: {}'.format(result5[1])

result_words5 = convert_to_words(result5[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words5)
print '====================='
print 'result words: {}'.format(result_words5)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words5)

print '\n'

print 'result stats'
print get_position_stats(result_words5)
"""

TEST5
pre smooth: [7 7 2 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 4 4 4 4 4 4 2 2 4 4
 1 0 0 0 3 0 0 0 5 0 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 6 3 6 6 6 6 6 6 6 6 6
 1 1 5 7]
result accuracy -101.599747597%
final result: [7 7 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 1 0 0 0 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 6 6 6 6 6 6 6 6 6
 1 1 0 0]
pre smooth words: ['OTHER', 'OTHER', 'your_closed_guard', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opp